In [4]:
df['label'].value_counts()  # -1 if could not be assigned to leaf -but not dropped

 0    78460
 1    54997
 4    26132
 2    19734
-1    11762
 5     9246
 3     4613
 6     4295
Name: label, dtype: int64

In [6]:
df['id_str'] = df['dr7objid']  # note - not iauname
del df['asset_id']
del df['dr7objid']

In [7]:
# convert to ortho labels
from zoobot.shared import label_metadata

current_labels = label_metadata.gz2_label_cols
ortho_labels = label_metadata.gz2_ortho_label_cols

count_renamer = dict(zip(current_labels, ortho_labels))

fraction_renamer = dict(zip([x + '_fraction' for x in current_labels], [x + '_fraction' for x in ortho_labels]))

renamer = count_renamer
renamer.update(fraction_renamer)
# renamer

df = df.rename(columns=renamer)

In [9]:
temp = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz2/images/' + df['filename']

import os
assert os.path.isfile(temp[0])

is_file = [os.path.isfile(loc) for loc in temp]
np.mean(is_file), len(is_file) -  np.sum(is_file)

(1.0, 0)

In [10]:
from PIL import Image

for loc in temp:
    if not Image.open(loc).size == (424, 424):
        print(loc)
        break

In [13]:
train_catalog_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz2/gz2_train_catalog.parquet'
test_catalog_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz2/gz2_test_catalog.parquet'

# zip not tar.gz as directly from Zenodo
images_loc = '/nvme1/scratch/walml/repos/pytorch-galaxy-datasets/roots/gz2/images_gz2.zip'

In [14]:
train_catalog, test_catalog = train_test_split(df, test_size=0.2, random_state=42)

train_catalog.to_parquet(train_catalog_loc, index=False)
test_catalog.to_parquet(test_catalog_loc, index=False)

In [15]:
import hashlib

for loc in [
    train_catalog_loc,
    test_catalog_loc,
    images_loc  # compressed from images above, takes 5 secs
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

f489c9ec7dcf8d99f728bd00ee00b1d0
8b2d74c93d35f86cc34f1d058b3b220b
bc647032d31e50c798770cf4430525c7
